Imports

In [280]:
import pandas as pd
import numpy as np

#### Read data and drop columns, fix names

In [281]:
df = pd.read_csv("gamesLog.csv")
df.drop(labels=["game_id","actor_id","date","time"],axis=1,inplace=True)
a = df.loc[:,"actor"] == "feyzi"
df.loc[a,"actor"] = "Feyzi"
a = df.loc[:,"actor"] == "5c651592-2d37-4622-ab5b-0ff4ca29f2f8"
df.loc[a,"actor"] = "Doruk"
df

,market,actor,price,round,type
0,spades,Feyzi,55,32,buy
1,spades,Feyzi,55,32,buy
2,spades,Feyzi,55,32,buy
3,diamonds,Doruk,34,32,sell
4,diamonds,Doruk,34,32,sell
...,...,...,...,...,...
1582,clubs,Doruk,40,1,sell
1583,spades,Doruk,40,1,sell
1584,hearts,Doruk,40,1,sell
1585,diamonds,Doruk,40,1,sell


# Summary stats

Some useful stats

In [282]:
summary = df[["market","actor","type"]].apply(pd.Series.value_counts).copy()
summary

,market,actor,type
Ben,NaN,162.0,NaN
Doruk,NaN,608.0,NaN
Feyzi,NaN,444.0,NaN
Isolde,NaN,199.0,NaN
Luke,NaN,174.0,NaN
buy,NaN,NaN,670.0
clubs,485.0,NaN,NaN
diamonds,575.0,NaN,NaN
hearts,283.0,NaN,NaN
sell,NaN,NaN,917.0


#### Calculate own R32 loss

In [283]:
round_32 = df[(df["market"]=="clubs") & (df["round"] == 32)].copy()
print(round_32.price.sum() /100, "$ loss")
print(round_32.shape[0], " trades")

27.76 $ loss
104  trades


### Contract balances for each player

DFs for all players

In [284]:
Isolde = df[["market","actor","type","price"]][df.actor=="Isolde"]
Luke   = df[["market","actor","type","price"]][df.actor=="Luke"]
Feyzi  = df[["market","actor","type","price"]][df.actor=="Feyzi"]
Ben    = df[["market","actor","type","price"]][df.actor=="Ben"]
Doruk  = df[["market","actor","type","price"]][df.actor=="Doruk"]

players = [Isolde, Luke, Feyzi, Ben, Doruk]

In [285]:
contracts_dict = {
    "Player": ["Isolde","Luke","Feyzi","Ben","Doruk"],
    "hearts": [0,0,0,0,0],
    "diamonds":[0,0,0,0,0],
    "clubs":[0,0,0,0,0],
    "spades":[0,0,0,0,0],
    "total_absolute" : [0,0,0,0,0],
    "total_net": [0,0,0,0,0]
}
contracts_df = df.from_dict(contracts_dict)
contracts_df.set_index("Player",inplace=True)
contracts_df

,hearts,diamonds,clubs,spades,total_absolute,total_net
Player,,,,,,
Isolde,0,0,0,0,0,0
Luke,0,0,0,0,0,0
Feyzi,0,0,0,0,0,0
Ben,0,0,0,0,0,0
Doruk,0,0,0,0,0,0


In [286]:
def get_net_contract(df,contract):
    return df[(df["type"] == "buy") & (df["market"] == contract)].shape[0] - df[(df["type"] == "sell") & (df["market"] == contract)].shape[0]

def calc_contract_balances(df):
    contracts= ["hearts","diamonds","clubs","spades"]
    balances = []
    for i in range(len(contracts)):
        balances.append(get_net_contract(df,contracts[i]))
    return balances

contracts= ["hearts","diamonds","clubs","spades"]

# Populate table
contracts_df.loc["Isolde",contracts] = calc_contract_balances(Isolde)
contracts_df.loc["Luke",contracts] = calc_contract_balances(Luke)
contracts_df.loc["Feyzi",contracts] = calc_contract_balances(Feyzi)
contracts_df.loc["Ben",contracts] = calc_contract_balances(Ben)
contracts_df.loc["Doruk",contracts] = calc_contract_balances(Doruk)


# Add market maker balances
contracts_df.loc["Isolde","hearts"] = -contracts_df.hearts.sum()
contracts_df.loc["Luke","diamonds"] = -contracts_df.diamonds.sum()
contracts_df.loc["Feyzi","clubs"] = -contracts_df.clubs.sum()
contracts_df.loc["Ben","spades"] = -contracts_df.spades.sum()

contract_balances_df = contracts_df.copy()

In [287]:
contracts_df.loc[:,"total_net"] = [26,143,419,-44,-544]
contracts_df.loc[:,"total_absolute"] = [132,143,419,144,544]

In [288]:
import seaborn as sns

cm = sns.light_palette("seagreen", as_cmap=True)

s = contracts_df.style.background_gradient(cmap=cm)
s

,hearts,diamonds,clubs,spades,total_absolute,total_net
Player,,,,,,
Isolde,65,-26,-27,14,132,26
Luke,10,99,15,19,143,143
Feyzi,49,104,177,89,419,419
Ben,2,30,18,-94,144,-44
Doruk,-126,-207,-183,-28,544,-544


#### Average prices and trade counts for each contract

In [290]:
average_buys = np.zeros(4)
average_sells = np.zeros(4)
average_spreads = np.zeros(4)
buy_count = np.zeros(4)
sell_count = np.zeros(4)

contracts= ["hearts","diamonds","clubs","spades"]

for i in range(4):
    average_buys[i] = round(df[(df.market==contracts[i]) & (df.type == "buy")].price.mean(),2)
    average_sells[i] = round(df[(df.market==contracts[i]) & (df.type == "sell")].price.mean(),2)
    average_spreads[i] = average_buys[i] - average_sells[i]
    buy_count[i] = round(df[(df.market==contracts[i]) & (df.type == "buy")].shape[0],2)
    sell_count[i]= round(df[(df.market==contracts[i]) & (df.type == "sell")].shape[0],2)




print(contracts)
print(buy_count)
print(average_buys)
print(average_sells)
print(sell_count)
print(average_spreads)



['hearts', 'diamonds', 'clubs', 'spades']
[109. 238. 154. 169.]
[25.53 22.97 20.47 37.54]
[21.48 27.89 17.64 28.97]
[174. 337. 331.  75.]
[ 4.05 -4.92  2.83  8.57]


In [293]:
# Put in dataframe
data = []

for i in range(4):
    data.append([contracts[i], buy_count[i], sell_count[i], average_buys[i]/100, average_sells[i]/100])                

    contracts_df = pd.DataFrame(data,columns=["Contract","#buys","#sells","Average buy price","Average sell price"])

    
    
contracts_df

,Contract,#buys,#sells,Average buy price,Average sell price
0,hearts,109.0,174.0,0.2553,0.2148
1,diamonds,238.0,337.0,0.2297,0.2789
2,clubs,154.0,331.0,0.2047,0.1764
3,spades,169.0,75.0,0.3754,0.2897


In [294]:
new_row = {
    'Contract': "All", '#buys':670, '#sells':917, 'Average buy price':0.265,
       'Average sell price':0.231
}

contracts_df = contracts_df.append( new_row, ignore_index=True)

C:\Users\feyzi\AppData\Local\Temp/ipykernel_19704/2205514523.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contracts_df = contracts_df.append( new_row, ignore_index=True)


In [295]:
contracts_df

,Contract,#buys,#sells,Average buy price,Average sell price
0,hearts,109.0,174.0,0.2553,0.2148
1,diamonds,238.0,337.0,0.2297,0.2789
2,clubs,154.0,331.0,0.2047,0.1764
3,spades,169.0,75.0,0.3754,0.2897
4,All,670.0,917.0,0.2650,0.2310


In [296]:
contracts_df.style.background_gradient(cmap='YlOrRd', axis=0)

,Contract,#buys,#sells,Average buy price,Average sell price
0,hearts,109.000000,174.000000,0.255300,0.214800
1,diamonds,238.000000,337.000000,0.229700,0.278900
2,clubs,154.000000,331.000000,0.204700,0.176400
3,spades,169.000000,75.000000,0.375400,0.289700
4,All,670.000000,917.000000,0.265000,0.231000


### Cash balances for each player
##### Problem here! add market maker balances

In [135]:
players = [Isolde, Luke, Feyzi, Ben, Doruk]

In [149]:
def calc_balance(df):
    balance = 0
    balance -= df[df["type"] == "buy"]["price"].sum()
    balance += df[df["type"] == "sell"]["price"].sum()
    return round(balance/100,2)

balances = np.zeros(5)
for i in range(len(players)):
    balances[i] = calc_balance(players[i])
balances

array([  0.18, -11.47, -61.31,  -4.14, 110.74])

### Need to add marketmakers revenue 

In [150]:
maker_sums = np.zeros(5)

for i in range(4):
    temp = df[(df["market"]== contracts[i]) & (df["type"] == "buy")].price.sum()
    temp -= df[(df["market"]== contracts[i]) & (df["type"] == "sell")].price.sum()
    maker_sums[i] =  round(temp/100,2)

maker_sums[4] = 0
maker_sums

array([ -9.54, -39.32, -26.85,  41.71,   0.  ])

In [151]:
net_balances = np.zeros(5)
for i in range(4):
    net_balances[i] = round(maker_sums[i] + balances[i],2)
    
net_balances[4] = balances[4]
net_balances

array([ -9.36, -50.79, -88.16,  37.57, 110.74])

In [152]:
grand_final = np.zeros(5)
for i in range(5):
    grand_final[i] = round(contract_balances_df.iloc[i,3] + net_balances[i],2)

grand_final

array([  4.64, -31.79,   0.84, -56.43,  82.74])

### Form into table 

In [162]:
balances_dict = {
    "Player": ["Isolde","Luke","Feyzi","Ben","Doruk"],
    "Net Buy/Sell Balance": [0,0,0,0,0],
    "Market Maker Revenue":[0,0,0,0,0],
    "Balance":[0,0,0,0,0],
    "Final Balance":[0,0,0,0,0],
}
balances_df = df.from_dict(balances_dict)
balances_df.set_index("Player",inplace=True)
balances_df

,Net Buy/Sell Balance,Market Maker Revenue,Balance,Final Balance
Player,,,,
Isolde,0,0,0,0
Luke,0,0,0,0
Feyzi,0,0,0,0
Ben,0,0,0,0
Doruk,0,0,0,0


#### Add data and format

In [165]:

balances_df.loc[:,"Net Buy/SellBalance"] = balances
balances_df.loc[:,"Market Maker Revenue"] = maker_sums
balances_df.loc[:,"Balance"] = net_balances
balances_df.loc[:,"Final Balance"] = grand_final

balances_df


,Net Buy/Sell Balance,Market Maker Revenue,Balance,Final Balance,Net Buy/Sell_Balance,Net Buy/SellBalance
Player,,,,,,
Isolde,0,-9.54,-9.36,4.64,0.18,0.18
Luke,0,-39.32,-50.79,-31.79,-11.47,-11.47
Feyzi,0,-26.85,-88.16,0.84,-61.31,-61.31
Ben,0,41.71,37.57,-56.43,-4.14,-4.14
Doruk,0,0.00,110.74,82.74,110.74,110.74


In [166]:
balances_df.style.background_gradient(cmap='YlGn', axis=0)

,Net Buy/Sell Balance,Market Maker Revenue,Balance,Final Balance,Net Buy/Sell_Balance,Net Buy/SellBalance
Player,,,,,,
Isolde,0,-9.540000,-9.360000,4.640000,0.180000,0.180000
Luke,0,-39.320000,-50.790000,-31.790000,-11.470000,-11.470000
Feyzi,0,-26.850000,-88.160000,0.840000,-61.310000,-61.310000
Ben,0,41.710000,37.570000,-56.430000,-4.140000,-4.140000
Doruk,0,0.000000,110.740000,82.740000,110.740000,110.740000


### Number of trades by player 

In [212]:
trade_dict = {
    "Player": ["Isolde","Luke","Feyzi","Ben","Doruk","All"],
    "#Buy_Orders": [0,0,0,0,0,0],
    "#Sell_Orders":[0,0,0,0,0,0],
    "Total_Orders":[0,0,0,0,0,0],
}

trades_df = df.from_dict(trade_dict)
trades_df

,Player,#Buy_Orders,#Sell_Orders,Total_Orders
0,Isolde,0,0,0
1,Luke,0,0,0
2,Feyzi,0,0,0
3,Ben,0,0,0
4,Doruk,0,0,0
5,All,0,0,0


In [213]:
temp_df = df.drop(["market", "round","price"],axis=1)
temp_df

,actor,type
0,Feyzi,buy
1,Feyzi,buy
2,Feyzi,buy
3,Doruk,sell
4,Doruk,sell
...,...,...
1582,Doruk,sell
1583,Doruk,sell
1584,Doruk,sell
1585,Doruk,sell


In [214]:
buys_by_player = np.zeros(6)
sells_by_player = np.zeros(6)
totals_by_player = np.zeros(6)
for i in range(5):
    buys_by_player[i] = temp_df[ ( temp_df["actor"] == actors[i]) & (temp_df["type"] == "buy")].shape[0]
    sells_by_player[i] = temp_df[ ( temp_df["actor"] == actors[i]) & (temp_df["type"] == "sell")].shape[0]
    totals_by_player[i] = buys_by_player[i]  + sells_by_player[i]

buys_by_player[5] = buys_by_player.sum()
sells_by_player[5] = sells_by_player.sum()
totals_by_player[5] = totals_by_player.sum()

In [218]:
trades_df.loc[:,"#Buy_Orders"] = buys_by_player
trades_df.loc[:,"#Sell_Orders"] = sells_by_player
trades_df.loc[:,"Total_Orders"] = totals_by_player

In [219]:
trades_df

,Player,#Buy_Orders,#Sell_Orders,Total_Orders
0,Isolde,80.0,119.0,199.0
1,Luke,109.0,65.0,174.0
2,Feyzi,343.0,101.0,444.0
3,Ben,106.0,56.0,162.0
4,Doruk,32.0,576.0,608.0
5,All,670.0,917.0,1587.0


In [245]:
trades_df.loc[0:5,:].style.background_gradient(cmap='OrRd', axis=0)

,Player,#Buy_Orders,#Sell_Orders,Total_Orders
0,Isolde,80.000000,119.000000,199.000000
1,Luke,109.000000,65.000000,174.000000
2,Feyzi,343.000000,101.000000,444.000000
3,Ben,106.000000,56.000000,162.000000
4,Doruk,32.000000,576.000000,608.000000
5,All,670.000000,917.000000,1587.000000
